In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, initializers, backend as K
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import shutil
from tensorflow.keras.saving import register_keras_serializable


In [14]:
@register_keras_serializable(package="Custom")
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

    def get_config(self):
        return super(Length, self).get_config()
@tf.keras.saving.register_keras_serializable(package="Custom", name="squash")
def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors
@tf.keras.saving.register_keras_serializable(package="Custom")
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        
        self.W = self.add_weight(
            shape=[1, self.input_num_capsule, self.num_capsule, self.dim_capsule, self.input_dim_capsule],
            initializer=initializers.glorot_uniform(),
            name='W'
        )
        self.built = True

    def call(self, inputs):
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)
        W_tiled = K.tile(self.W, [K.shape(inputs)[0], 1, 1, 1, 1])
        
        inputs_hat = tf.squeeze(tf.matmul(W_tiled, inputs_expand, transpose_b=True), axis=-1)
        b = tf.zeros(shape=[K.shape(inputs)[0], self.input_num_capsule, self.num_capsule])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            c_expand = K.expand_dims(c, -1)
            outputs = squash(tf.reduce_sum(inputs_hat * c_expand, axis=1))
            
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=-1)
        
        return outputs
@tf.keras.saving.register_keras_serializable(package="Custom", name="margin_loss")
def margin_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=2)
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))


In [15]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from pathlib import Path

# Configuration
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
DATA_DIR = Path("D:/Major Project/Rasp/Data")
TEST_DIR = r"D:\Major Project\Rasp\Data\test"

# Load the trained model
model = load_model("drowsiness_model_mobilecap_lesser.keras", custom_objects={"CapsuleLayer": CapsuleLayer,"Length":Length,"margin_loss":margin_loss})

In [16]:
# Define test image generator
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load test images
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Ensure binary classification mode
    shuffle=False
)


Found 2819 images belonging to 2 classes.


In [17]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_gen)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 35s 372ms/step - accuracy: 0.7473 - loss: 0.1497
Test Loss: 0.1159
Test Accuracy: 0.8237


In [19]:
from sklearn.metrics import classification_report

# Get predictions (probabilities)
y_pred = np.argmax(model.predict(test_gen), axis=1)
y_true = test_gen.classes

print("\nTest Metrics:")
print(f"Accuracy: {np.mean(y_true == y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Not Drowsy', 'Drowsy']))


89/89 ━━━━━━━━━━━━━━━━━━━━ 35s 387ms/step

Test Metrics:
Accuracy: 0.8237

Classification Report:
              precision    recall  f1-score   support

  Not Drowsy       0.82      0.86      0.84      1507
      Drowsy       0.83      0.79      0.81      1312

    accuracy                           0.82      2819
   macro avg       0.82      0.82      0.82      2819
weighted avg       0.82      0.82      0.82      2819

